
# LeanTrader — XAUUSD Parameter Sweep

This notebook searches **ADX threshold**, **ATR stop/take multipliers**, and **FVG lookback** for the XAUUSD strategy.

It reports:
- Overall **Sharpe** and **Max Drawdown**
- **Per-session** scores (Asia, London, NY)
- Saves the **best params** to `data/tuned/xauusd_params.yaml`


In [ ]:

from pathlib import Path

import pandas as pd
import yaml

from leantrader.backtest.engine import backtest
from leantrader.backtest.metrics import max_drawdown, sharpe
from leantrader.data.feeds import get_ohlc_csv, resample_frames
from leantrader.dsl.compiler import compile_strategy, load_strategy
from leantrader.features.ta import adx, fvg_score, rsi
from leantrader.sessions.manager import which_session

REPO_ROOT = Path.cwd().resolve()
DATA_DIR = REPO_ROOT / "data" / "ohlc"
TUNE_DIR = REPO_ROOT / "data" / "tuned"
TUNE_DIR.mkdir(parents=True, exist_ok=True)

PAIR = "XAUUSD"
m15_path = DATA_DIR / f"{PAIR}_M15.csv"
assert m15_path.exists(), f"Missing {m15_path}"
df_m15 = get_ohlc_csv(str(m15_path))

# Build frames aligned to M15
frames = resample_frames(df_m15)

def engineer_param(df: pd.DataFrame, fvg_lb: int = 3) -> pd.DataFrame:
    out = df.copy()
    out["rsi_14"] = rsi(out["close"], 14)
    out["adx_14"] = adx(out, 14)
    out["fvg_score"] = fvg_score(out, fvg_lb)
    out["ms_state"] = "flat"
    swing_up = (out["high"] > out["high"].shift(1)) & (out["low"] > out["low"].shift(1))
    swing_dn = (out["high"] < out["high"].shift(1)) & (out["low"] < out["low"].shift(1))
    out.loc[swing_up, "ms_state"] = "bull"
    out.loc[swing_dn, "ms_state"] = "bear"
    out["rsi_div"] = (out["close"].diff() < 0).astype(int) * ((out["rsi_14"].diff() > 0).astype(int)) -                      (out["close"].diff() > 0).astype(int) * ((out["rsi_14"].diff() < 0).astype(int))
    return out

# Load the base XAU strategy
spec_path = REPO_ROOT / "src" / "leantrader" / "dsl" / "examples" / "xauusd_master.yaml"
assert spec_path.exists(), f"Missing {spec_path}"
base_spec = load_strategy(str(spec_path))

def spec_with_params(spec, adx_thr: int, atr_stop: float, r_mult: float):
    import copy
    s = copy.deepcopy(spec)
    for sig in s.signals:
        # replace ADX threshold and risk params in-place by string substitution in the feature expression
        for cond in sig.entry:
            if cond.feature.startswith("adx_14"):
                cond.feature = f"adx_14 > {adx_thr}"
        sig.stop = f"atr_14 * {atr_stop}"
        sig.take = f"R:{r_mult}"
    return s

param_space = {
    "adx_thr": [16, 18, 20, 22, 25],
    "atr_stop": [1.4, 1.6, 1.8],
    "r_mult": [2.0, 2.2, 2.5],
    "fvg_lb": [2, 3, 4]
}

def equity_by_session(eq: pd.Series) -> dict:
    # Assign session labels by index timestamp and split equity
    labs = [which_session(pd.Timestamp(t)) for t in eq.index]
    df = pd.DataFrame({"eq": eq.values, "session": labs}, index=eq.index)
    parts = {s: df[df["session"]==s]["eq"] for s in ["asia","london","ny"]}
    return parts

def score_equity(eq: pd.Series) -> float:
    # combined score: Sharpe - |MaxDD|
    return float(sharpe(eq)) - abs(float(max_drawdown(eq)))

best = None
rows = []
for adx_thr in param_space["adx_thr"]:
    for atr_stop in param_space["atr_stop"]:
        for r_mult in param_space["r_mult"]:
            for fvg_lb in param_space["fvg_lb"]:
                eng = {k: engineer_param(v, fvg_lb=fvg_lb) for k,v in frames.items()}
                spec = spec_with_params(base_spec, adx_thr=adx_thr, atr_stop=atr_stop, r_mult=r_mult)
                strat = compile_strategy(spec)
                sigs = strat(eng)
                m15 = eng["M15"]
                sigs["price"] = m15["close"].reindex(sigs.index, method="ffill")
                eq = backtest(m15, sigs, risk_cfg=None)
                if len(eq)==0: continue
                sc = score_equity(eq)
                parts = equity_by_session(eq)
                row = {
                    "adx_thr": adx_thr,
                    "atr_stop": atr_stop,
                    "r_mult": r_mult,
                    "fvg_lb": fvg_lb,
                    "score": sc,
                    "sharpe": float(sharpe(eq)),
                    "maxdd": float(max_drawdown(eq)),
                    "sh_asia": float(sharpe(parts["asia"])) if len(parts["asia"])>10 else 0.0,
                    "sh_london": float(sharpe(parts["london"])) if len(parts["london"])>10 else 0.0,
                    "sh_ny": float(sharpe(parts["ny"])) if len(parts["ny"])>10 else 0.0,
                }
                rows.append(row)
                if best is None or sc > best["score"]:
                    best = row

import pandas as pd

res_df = pd.DataFrame(rows).sort_values("score", ascending=False)
best


In [ ]:

# Display top 15 configs
import pandas as pd

res_df.head(15)


In [ ]:

# Save best params to YAML for production loading
best_path = TUNE_DIR / "xauusd_params.yaml"
with open(best_path, "w", encoding="utf-8") as f:
    yaml.safe_dump({
        "pair": PAIR,
        "best": best,
        "params_sorted": res_df.head(50).to_dict(orient="records")
    }, f, sort_keys=False)
best_path



## Next
- Load `data/tuned/xauusd_params.yaml` in your live runner and adjust the strategy thresholds at startup.
- Paste the prompts from **COPILOT_PLAYBOOK_FINAL.md** to wire live feeds and execution.
